# Quantum approximate optimization algorithm

The quantum approximate optimization algorithm (QAOA) is a shallow-circuit variational algorithm for gate-model quantum computers that was inspired by quantum annealing. We discretize the adiabatic pathway in some $p$ steps, where $p$ influences precision. Each discrete time step $i$ has two parameters, $\beta_i, \gamma_i$. The classical variational algorithms does an optimization over these parameters based on the observed energy at the end of a run on the quantum hardware.

More formally, we want to discretize the time-dependent $H(t)=(1-t)H_0 + tH_1$ under adiabatic conditions. We achieve this by Trotterizing the unitary. For instance, for time step $t_0$, we can split this unitary as $U(t_0) = U(H_0, \beta_0)U(H_1, \gamma_0)$. We can continue doing this for subsequent time steps, eventually splitting up the evolution to $p$ such chunks:

$$
U = U(H_0, \beta_0)U(H_1, \gamma_0)\ldots U(H_0, \beta_p)U(H_1, \gamma_p).
$$

At the end of optimizing the parameters, this discretized evolution will approximate the adiabatic pathway:

The Hamiltonian $H_0$ is often referred to as the driving or mixing Hamiltonian, and $H_1$ as the cost Hamiltonian. The simplest mixing Hamiltonian is $H_0 = -\sum_i \sigma^X_i$, the same as the initial Hamiltonian in quantum annealing. By alternating between the two Hamiltonian, the mixing Hamiltonian drives the state towards an equal superposition, whereas the cost Hamiltonian tries to seek its own ground state.


In [1]:
import itertools
import numpy as np
from qiskit import BasicAer, QuantumRegister, execute
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator, MatrixOperator
from qiskit.aqua import Operator, get_aer_backend
from qiskit.aqua.components.initial_states import Custom
from scipy.optimize import minimize
np.set_printoptions(precision=3, suppress=True)
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'qiskit.aqua.operators'

In [ ]:
def pauli_x(qubit, coeff):
    eye = np.eye((n_qubits)) 
    return Operator([[coeff, Pauli(np.zeros(n_qubits), eye[qubit])]])

def pauli_z(qubit, coeff):
    eye = np.eye((n_qubits))
    return Operator([[coeff, Pauli(eye[qubit], np.zeros(n_qubits))]])

def product_pauli_z(q1, q2, coeff):
    eye = np.eye((n_qubits))
    return Operator([[coeff, Pauli(eye[q1], np.zeros(n_qubits)) * Pauli(eye[q2], np.zeros(n_qubits))]])

In [ ]:
pauli_dict_mix = {
    'paulis': [{"coeff": {"imag": 0.0, "real": -1.0}, "label": "IX"},
              {"coeff": {"imag": 0.0, "real": -1.0}, "label": "XI"}]
}

pauli_dict_cost = {
    'paulis': [{"coeff": {"imag": 0.0, "real": -1.0}, "label": "ZZ"}]
}

pauli_dict_iden = {
    'paulis': [{"coeff": {"imag": 0.0, "real": 1.0}, "label": "II"}]
}

H_mix = qiskit.aqua.operators.WeightedPauliOperator.from_dict(pauli_dict_mix)
H_cost = WeightedPauliOperator.from_dict(pauli_dict_cost)
H_iden = WeightedPauliOperator.from_dict(pauli_dict_iden)

print(H_mix, H_cost, H_iden)

In [ ]:
def ham_mat(operator):
    hamiltonian = 0
    for weight, pauli in operator.paulis:
        hamiltonian += weight * pauli.to_spmatrix()
    return MatrixOperator(matrix=hamiltonian, z2_symmetries=operator.z2_symmetries, name=operator.name)

In [ ]:
n_qubits = 2

In [ ]:
identity = pauli_x(0, 0)

Hm = sum([pauli_x(i, -1) for i in range(n_qubits)], identity)
Hm.to_matrix()
print(Hm._matrix.toarray())
print(ham_mat(H_mix).dense_matrix)

Then we define the cost Hamiltonian $H_c=-\sigma^Z_1 \otimes \sigma^Z_2$:

In [ ]:
J = np.array([[0,1],[0,0]])

# itertools.product returns a list of all the pairs (i,j) lower than n_qubits
Hc = sum([product_pauli_z(i,j, -J[i,j])
             for i,j in itertools.product(range(n_qubits), repeat=2)], identity)
Hc.to_matrix()
print(Hc._matrix.toarray())
print(ham_mat(H_cost).dense_matrix)

In [ ]:
n_iter = 10
p = 2
beta = np.random.uniform(0, np.pi*2, p)
gamma = np.random.uniform(0, np.pi*2, p)

In [ ]:
init_state_vect = [1 for i in range(2**n_qubits)]
init_state = Custom(n_qubits, state_vector=init_state_vect)

In [ ]:
qr = QuantumRegister(n_qubits, name='q')
circuit_init = init_state.construct_circuit('circuit', qr)

In [ ]:
def evolve_dm(hamiltonian, angle, quantum_registers):
    return hamiltonian.evolve(state_in=None, evo_time=angle, evo_mode=None, num_time_slices=1,
                              quantum_registers=quantum_registers,
                              expansion_mode='suzuki',
                              expansion_order=3)

def evolve_un(hamiltonian, angle, quantum_registers):
    return hamiltonian.evolve(None, angle, 'circuit', 1,
                              quantum_registers=quantum_registers,
                              expansion_mode='suzuki',
                              expansion_order=3)

In [ ]:
def create_circuit_dm(beta_gamma):
    n = len(beta_gamma)//2
    beta = beta_gamma[:n]
    gamma = beta_gamma[n:]
    circuit_evolv = sum([evolve_dm(H_cost, beta[i],qr) + evolve_dm(H_mix, gamma[i], qr) 
                        for i in range(p)], evolve_dm(H_iden, 0, qr))
    circuit = circuit_init + circuit_evolv
    return circuit

def create_circuit_un(beta_gamma):
    n = len(beta_gamma)//2
    beta = beta_gamma[:n]
    gamma = beta_gamma[n:]
    circuit_evolv = sum([evolve_un(Hc, beta[i], qr) + evolve_un(Hm, gamma[i], qr)
                            for i in range(p)], evolve_un(identity, 0, qr))
    circuit = circuit_init + circuit_evolv
    return circuit

In [ ]:
def densemat(vec):
        dens = vec.copy()
        trans = []
        import itertools
        bina1 = [x for x in itertools.product(
            [0, 1], repeat=n_qubits)]
        bina2 = [''.join(str(y) for y in x) for x in bina1]
        for idx in bina2:
            a = int(idx, 2)
            b = int(idx[::-1], 2)
            if a not in trans and b not in trans:
                trans.append(a)
                trans.append(b)
                dens[a], dens[b] = dens[b], dens[a]
        densitymatrix = np.outer(dens, np.conj(dens))
        return densitymatrix

In [ ]:
def evaluate_circuit_dm(beta_gamma):
    circuit = create_circuit_dm(beta_gamma)
    result = execute(circuit, backend=BasicAer.get_backend('dm_simulator')).result()
    dm = result['results'][0]['data']['densitymatrix']
    Hc_mat = ham_mat(H_cost)
    return np.real(np.trace(np.matmul(dm, Hc_mat.dense_matrix)))

def evaluate_circuit_un(beta_gamma):
    circuit = create_circuit_un(beta_gamma)
    result = execute(circuit, backend=BasicAer.get_backend('qasm_simulator')).result()
    st =  np.reshape(result['results'][0]['data']['statevector'], 2 ** n_qubits)
    dm = densemat(st) 
    Hc.to_matrix()
    return np.real(np.trace(np.matmul(dm, Hc._matrix.toarray())))

In [ ]:
evaluate_circuit_un(np.concatenate([beta, gamma]))

In [ ]:
evaluate_circuit_dm(np.concatenate([beta, gamma]))

Finally, we optimize the angles:

In [ ]:
result_un = minimize(evaluate_circuit_un, np.concatenate([beta, gamma]), method='L-BFGS-B')
print(result_un)

In [ ]:
result_dm = minimize(evaluate_circuit_dm, np.concatenate([beta, gamma]), method='L-BFGS-B')
print(result_dm)